# 2.3 自动求梯度

在深度学习中，我们经常需要对函数求梯度（gradient）。本节将介绍如何使用PyTorch 提供的`autograd`模块来自动求梯度。如果对本节中的数学概念（如梯度）不是很熟悉，可以参阅附录中[“数学基础”](../chapter11_appendix/11.02_math.ipynb)一节。

In [1]:
import torch

print(torch.__version__)

1.3.0+cpu


## 2.3.1 简单例子

我们先看一个简单例子：对函数 $y = 2\boldsymbol{x}^{\top}\boldsymbol{x}$ 求关于列向量 $\boldsymbol{x}$ 的梯度。我们先创建变量`x`，并赋初值。

In [2]:
x = torch.reshape(torch.arange(4, dtype=torch.float64), (4, 1))
x

tensor([[0.],
        [1.],
        [2.],
        [3.]], dtype=torch.float64)

为了求有关变量`x`的梯度，我们需要先调用`attach_grad`函数来申请存储梯度所需要的内存。

In [3]:
x.requires_grad_(True)

tensor([[0.],
        [1.],
        [2.],
        [3.]], dtype=torch.float64, requires_grad=True)

下面定义有关变量`x`的函数。

In [4]:
y = 2 * torch.mm(x.T, x)

由于`x`的形状为（4, 1），`y`是一个标量。接下来我们可以通过调用`backward`函数自动求梯度。需要注意的是，如果`y`不是一个标量，则该函数需要一个`Tensor`作为参数，详细参加这个[教程](../../TestEverything/DeepLearningWithPyTorch-A60MinuteBlitz/02autograd_tutorial.ipynb)。

In [5]:
#y.backward()
torch.autograd.backward(y)
print(y)

tensor([[28.]], dtype=torch.float64, grad_fn=<MulBackward0>)


函数 $y = 2\boldsymbol{x}^{\top}\boldsymbol{x}$ 关于$\boldsymbol{x}$ 的梯度应为$4\boldsymbol{x}$。现在我们来验证一下求出来的梯度是正确的。

In [6]:
assert (x.grad - 4 * x).norm().item() == 0
x.grad

tensor([[ 0.],
        [ 4.],
        [ 8.],
        [12.]], dtype=torch.float64)

## 2.3.2 自动清零

再次开始一个求导过程之前，需要将已有的梯度清零，否则它将以原值参加新的运算，而导致结果不正确。

In [8]:
x.grad.data.zero_()
x.grad

tensor([[0.],
        [0.],
        [0.],
        [0.]], dtype=torch.float64)

## 2.3.3 对Python控制流求梯度

使用PyTorch的一个便利之处是，即使函数的计算图包含了Python的控制流（如条件和循环控制），我们也有可能对变量求梯度。

考虑下面程序，其中包含Python的条件和循环控制。需要强调的是，这里循环（while循环）迭代的次数和条件判断（if语句）的执行都取决于输入`a`的值。

In [8]:
def f(a):
    b = a * 2
    while b.norm().item() < 1000:
        b = b * 2
    if b.sum().item() > 0:
        c = b
    else:
        c = 100 * b
    return c

In [9]:
a = torch.normal(2, 3, size=(1, 1))
print(a)
a.requires_grad_(True)
with torch.set_grad_enabled(True):
    c = f(a)
c.backward()
print(a.grad)

tensor([[-4.7924]])
tensor([[25600.]])


[`torch.set_grad_enabled()`](https://pytorch.org/docs/stable/autograd.html#torch.autograd.set_grad_enabled)（[中文参考文档](https://pytorch.apachecn.org/docs/1.2/autograd.html)）是上下文管理器，其作用是打开/关闭梯度计算。

我们来分析一下上面定义的`f`函数。事实上，给定任意输入`a`，其输出必然是 `f(a) = x * a`的形式，其中标量系数`x`的值取决于输入`a`。由于`c = f(a)`有关`a`的梯度为`x`，且值为`c / a`，我们可以像下面这样验证对本例中控制流求梯度的结果的正确性。

In [10]:
a.grad == c / a

tensor([[True]])

# 2.3 自动求梯度
## 2.3.1 概念
上一节介绍的`Tensor`是这个包的核心类，如果将其属性`.requires_grad`设置为`True`，它将开始追踪(track)在其上的所有操作。完成计算后，可以调用`.backward()`来完成所有梯度计算。此`Tensor`的梯度将累积到`.grad`属性中。
> 注意在调用`.backward()`时，如果`Tensor`是标量，则不需要为`backward()`指定任何参数；否则，需要指定一个求导变量。

如果不想要被继续追踪，可以调用`.detach()`将其从追踪记录中分离出来，这样就可以防止将来的计算被追踪。此外，还可以用`with torch.no_grad()`将不想被追踪的操作代码块包裹起来，这种方法在评估模型的时候很常用，因为在评估模型时，我们并不需要计算可训练参数（`requires_grad=True`）的梯度。

`Function`是另外一个很重要的类。`Tensor`和`Function`互相结合就可以构建一个记录有整个计算过程的非循环图。每个`Tensor`都有一个`.grad_fn`属性，该属性即创建该`Tensor`的`Function`（除非用户创建的`Tensor`s时设置了`grad_fn=None`）。

下面通过一些例子来理解这些概念。

## 2.3.2 `Tensor`

In [ ]:
x = torch.ones(2, 2, requires_grad=True)
print(x)
print(x.grad_fn)

In [ ]:
y = x + 2
print(y)
print(y.grad_fn)

注意x是直接创建的，所以它没有`grad_fn`, 而y是通过一个加法操作创建的，所以它有一个为`<AddBackward>`的`grad_fn`。

In [ ]:
print(x.is_leaf, y.is_leaf)

In [ ]:
z = y * y * 3
out = z.mean()
print(z, out)

通过`.requires_grad_()`来用in-place的方式改变`requires_grad`属性：

## 小结

* PyTorch提供`autograd`模块来自动化求导过程，但是每个独立的梯度求取过程之前要进行梯度清零。
* PyTorch的`autograd`模块可以对一般的命令式程序进行求导。
* PyTorch提供一个梯度计算的开关`set_grad_enabled()`，可以被用在有条件的梯度计算之场景下。

## 练习

* 在本节对控制流求梯度的例子中，把变量`a`改成一个随机向量或矩阵。此时计算结果`c`不再是标量，运行结果将有何变化？该如何分析该结果？
* 重新设计一个对控制流求梯度的例子。运行并分析结果。

## 扫码直达[知乎专栏](https://zhuanlan.zhihu.com/unicom-d2l)

![](../img/zhihu.png)